In [ ]:
import torch 
import pandas as pd
from sklearn.preprocessing import OneHotEncoder as ohe
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv("../../data/raw/obesity_prediction.csv")

In [ ]:
df.head(3)

In [ ]:
enc = ohe(handle_unknown="warn",sparse_output=False).set_output(transform="pandas")

In [ ]:
enc.fit(np.asarray(df["Gender"]).reshape(-1,1))

In [ ]:
enc.categories_

In [ ]:
gender_enc_df = enc.transform(np.asarray(df["Gender"]).reshape(-1,1))

In [ ]:
gender_enc_df

In [ ]:
gender_enc_df.columns =["female","male"]

In [ ]:
gender_enc_df.head()

In [ ]:
enc2 = ohe(sparse_output=False,handle_unknown="ignore").set_output(transform="pandas")

In [ ]:
df_mtran = enc2.fit_transform(np.asarray(df["MTRANS"]).reshape(-1,1))

In [ ]:
df_mtran.head()

In [ ]:
enc2.categories_

In [ ]:
df_mtran.columns = ["automobile","bike","motorbike","public_transportation","walking"]

In [ ]:
df_mtran.head(3)

In [ ]:
df_mtran.head()

In [ ]:
gender_enc_df.head()

In [ ]:
df_x = pd.concat(objs=[gender_enc_df,df_mtran,df["Age"],df["Height"]],axis=1)

In [ ]:
df_x

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
y_enc = le.fit_transform(df["Obesity"])

In [ ]:
y_enc

In [ ]:
le.classes_

In [ ]:
Y = torch.tensor(y_enc)

In [ ]:
Y.shape

In [ ]:
scaler = StandardScaler().set_output(transform="pandas")

In [ ]:
df_age_scaled = scaler.fit_transform(np.asarray(df["Age"]).reshape(-1,1))
df_height_scaled = scaler.fit_transform(np.asarray(df["Height"]).reshape(-1,1))

In [ ]:
df_height_scaled.columns =["height"]
df_age_scaled.columns = ["age"]

In [ ]:
df_height_scaled

In [ ]:
df_x = df_x.drop(labels=["Age","Height"],axis=1)
df_x.head()

In [ ]:
df_x = pd.concat(axis=1,objs=[df_x,df_age_scaled,df_height_scaled])

In [ ]:
df_x.head()

In [ ]:
class ffn(torch.nn.Module):
    
    def __init__(self,input_dims):
        super(ffn,self).__init__()    
        
        self.relu = torch.nn.ReLU()

        self.layer1 = torch.nn.Linear(input_dims,100)
        self.layer2 = torch.nn.Linear(100,50)
        self.layer3 = torch.nn.Linear(50,7)
    def forward(self,input):
        
        out = self.layer1(input)
        out = self.relu(self.layer2(out))
        out = self.relu(self.layer3(out))

        return out

In [ ]:
Y

In [ ]:
X = torch.tensor(np.asarray(df_x))

In [ ]:
X.shape

In [ ]:
df_x.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [ ]:
class data(torch.utils.data.Dataset):

    def __init__(self, X,y):
        self.x = X
        self.y = y
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        record = self.x[index]
        label = self.y[index]
        return record,label

In [ ]:
len(X_train[0])

In [ ]:
model = ffn(9)

In [ ]:
train_dataset = data(X_train,y_train)
test_dataset = data(X_test,y_test)

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=32,shuffle=True)

In [121]:
model.train()
epochs = 50

optim = torch.optim.AdamW(model.parameters(),lr = 0.001)

loss_fn = torch.nn.CrossEntropyLoss()
y_true = []
y_pred = []

for epoch in range(epochs):
    loss_per_epoch = 0
    for batch in train_loader:
        
        inputs,labels = batch
        optim.zero_grad()
        
        outputs = model(inputs.float())
        loss    = loss_fn(outputs,labels)
       
        _,preds = torch.max(outputs,1)
    
        y_pred.extend(preds.detach().tolist())
        y_true.extend(labels.detach().tolist())
        loss.backward()
        optim.step()
        
        loss_per_epoch = loss.item()
    
    print(f"Epoch: {epoch + 1}   Loss: {loss_per_epoch}")
    print(classification_report(y_true,y_pred))

Epoch: 1   Loss: 0.9716035723686218
              precision    recall  f1-score   support

           0       0.40      0.67      0.50       182
           1       0.41      0.39      0.40       184
           2       0.00      0.00      0.00       237
           3       0.72      0.92      0.81       203
           4       0.75      0.93      0.83       209
           5       0.43      0.41      0.42       196
           6       0.47      0.52      0.49       203

    accuracy                           0.54      1414
   macro avg       0.45      0.55      0.49      1414
weighted avg       0.45      0.54      0.49      1414

Epoch: 2   Loss: 0.9518008828163147
              precision    recall  f1-score   support

           0       0.40      0.68      0.51       364
           1       0.42      0.39      0.40       368
           2       0.00      0.00      0.00       474
           3       0.72      0.92      0.81       406
           4       0.75      0.94      0.83       418
      

/home/mgarcia/github/obese_classifier/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mgarcia/github/obese_classifier/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mgarcia/github/obese_classifier/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

Epoch: 6   Loss: 0.949380099773407
              precision    recall  f1-score   support

           0       0.41      0.68      0.51      1092
           1       0.43      0.38      0.41      1104
           2       0.00      0.00      0.00      1422
           3       0.72      0.92      0.81      1218
           4       0.74      0.94      0.83      1254
           5       0.43      0.44      0.44      1176
           6       0.47      0.53      0.50      1218

    accuracy                           0.54      8484
   macro avg       0.46      0.55      0.50      8484
weighted avg       0.45      0.54      0.49      8484

Epoch: 7   Loss: 0.9467708468437195
              precision    recall  f1-score   support

           0       0.41      0.68      0.51      1274
           1       0.43      0.38      0.41      1288
           2       0.00      0.00      0.00      1659
           3       0.72      0.92      0.81      1421
           4       0.74      0.94      0.83      1463
       

/home/mgarcia/github/obese_classifier/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mgarcia/github/obese_classifier/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mgarcia/github/obese_classifier/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

Epoch: 11   Loss: 0.9422516822814941
              precision    recall  f1-score   support

           0       0.41      0.68      0.51      2002
           1       0.44      0.38      0.41      2024
           2       0.00      0.00      0.00      2607
           3       0.72      0.92      0.81      2233
           4       0.74      0.94      0.83      2299
           5       0.44      0.44      0.44      2156
           6       0.47      0.53      0.50      2233

    accuracy                           0.54     15554
   macro avg       0.46      0.56      0.50     15554
weighted avg       0.45      0.54      0.49     15554

Epoch: 12   Loss: 0.941903293132782
              precision    recall  f1-score   support

           0       0.41      0.68      0.51      2184
           1       0.44      0.38      0.41      2208
           2       0.00      0.00      0.00      2844
           3       0.72      0.92      0.81      2436
           4       0.74      0.94      0.83      2508
     

...

/home/mgarcia/github/obese_classifier/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mgarcia/github/obese_classifier/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mgarcia/github/obese_classifier/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

              precision    recall  f1-score   support

           0       0.41      0.68      0.51      8008
           1       0.44      0.38      0.41      8096
           2       0.00      0.00      0.00     10428
           3       0.72      0.92      0.81      8932
           4       0.74      0.94      0.83      9196
           5       0.43      0.44      0.44      8624
           6       0.47      0.52      0.49      8932

    accuracy                           0.55     62216
   macro avg       0.46      0.56      0.50     62216
weighted avg       0.45      0.55      0.49     62216

Epoch: 45   Loss: 0.9197322726249695
              precision    recall  f1-score   support

           0       0.41      0.68      0.51      8190
           1       0.44      0.38      0.41      8280
           2       0.00      0.00      0.00     10665
           3       0.72      0.92      0.81      9135
           4       0.74      0.94      0.83      9405
           5       0.43      0.44      0.

/home/mgarcia/github/obese_classifier/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mgarcia/github/obese_classifier/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mgarcia/github/obese_classifier/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

Epoch: 48   Loss: 0.9231583476066589
              precision    recall  f1-score   support

           0       0.41      0.68      0.51      8736
           1       0.44      0.38      0.41      8832
           2       0.00      0.00      0.00     11376
           3       0.72      0.92      0.81      9744
           4       0.74      0.94      0.83     10032
           5       0.43      0.45      0.44      9408
           6       0.47      0.52      0.49      9744

    accuracy                           0.55     67872
   macro avg       0.46      0.56      0.50     67872
weighted avg       0.45      0.55      0.49     67872

Epoch: 49   Loss: 0.9259502291679382
              precision    recall  f1-score   support

           0       0.41      0.68      0.51      8918
           1       0.44      0.38      0.41      9016
           2       0.00      0.00      0.00     11613
           3       0.72      0.92      0.81      9947
           4       0.74      0.94      0.83     10241
    

/home/mgarcia/github/obese_classifier/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mgarcia/github/obese_classifier/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mgarcia/github/obese_classifier/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [ ]:
model.eval()

with torch.no_grad():
    test_loss = 0
    for batch in test_loader:
        
        inputs,labels = batch
        outputs = model(inputs.float())
        loss = loss_fn(outputs,labels)
        test_loss = loss.item()

    print(f"Test Loss: {test_loss}")
    
    

In [ ]:
a = torch.randn(10,3)
print(a)
val, idx = torch.max(a,1)

In [ ]:
idx

In [ ]:
b = torch.randn(5)

In [ ]:
torch.max(b,0)


In [122]:
type(X_train)

torch.Tensor

In [128]:
dict = {0:0,1:0,2:0,3:0,4:0,5:0,6:0}

for cls in y_train:

    y = cls.item()

    dict[y] += 1

In [129]:
dict

{0: 182, 1: 184, 2: 237, 3: 203, 4: 209, 5: 196, 6: 203}